<a href="https://colab.research.google.com/github/akash166d/sparkNLP/blob/master/DoD_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Package/Lib

This is required to ensure the ubuntu related dependencies are up to date

In [1]:
! sudo apt-get update --fix-missing
! sudo apt-get upgrade --fix-missing

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,688 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:12 https://developer.download.nvidia.com/compute/machine-learning/repo

In [2]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.5.4

! pip install --user -U nltk

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)
     |████████████████████████████████| 215.7MB 77kB/s 
     |████████████████████████████████| 204kB 48.1MB/s 
     |████████████████████████████████| 133kB 12.9MB/s 
     |████████████████████████████████| 1.4MB 11.9MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434679 sha256=42921dff711793104ce35f0022af0219dcdd568a4887bc7fcf4596be378368b9
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  2.5.4
Apache Spark version;  2.4.4


# Mount Drive and read data

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Some of the sparkNLP annotators were having trouble reading spaces in path, hence used symbolic link to replace the path

In [5]:
! ln -s "/content/drive/My Drive" "/content/MyDrive"

In [6]:

song_data = spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/SparkNLP/song_2k.csv")


In [7]:
from pyspark.sql import functions as F
song_data = song_data.select(['Key', 'Lyric', 'Genre']).withColumnRenamed('Lyric','text')

# train, test = trainDataset.randomSplit(weights=[0.5, 0.5], seed=123)
song_data = song_data.limit(999)
print(song_data.count())

song_data = song_data.filter(song_data.text != '')
print(song_data.count())
song_data = song_data.filter((song_data.Genre == 'Rock') | (song_data.Genre == 'Hip Hop') | (song_data.Genre == 'Pop')  )
# song_data = song_data.filter(song_data.Genre != '')
print(song_data.count())
# song_data = song_data.filter(song_data.text != ' ')
# print(song_data.count())
# song_data = song_data.filter(song_data.Genre != ' ')
# print(song_data.count())
song_data.na.drop(subset=["text"])
print(song_data.count())
song_data.show()

999
999
915
915
+--------------------+--------------------+-------+
|                 Key|                text|  Genre|
+--------------------+--------------------+-------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|
|jamiroquai_Rock D...|And it's coming a...|    Pop|
|10000 maniacs_The...|These are. These ...|   Rock|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|
|black veil brides...|Have we begun to ...|   Rock|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|
|10000 maniacs_Acr...|Well they left th...|   Rock|
|10000 maniacs_Pla...|[ music: Dennis D...|   Rock|
|10000 maniacs_Rai...|On bended kneeI'v...|   Rock|
|twista_Back 2 School|[Tung Twista]. ba...|Hip Hop|
|10000 maniacs_Ant...|For whom do the b...|   Rock|
|10000 maniacs_All...|She walks alone o...|   Rock|
|10000 maniacs_Bac...|Jenny. Jenny you ...|   Rock|
|cyndi lauper_True...|You with the sad ...|    P

In [8]:
print(type(song_data))

<class 'pyspark.sql.dataframe.DataFrame'>


# Package Import

In [9]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from pyspark.sql import functions as F

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Vocab Reduction

Basic Data cleaning for NLP

include NER and filter out person and other such tag

In [10]:
assembler = DocumentAssembler().setInputCol('text').setOutputCol('document')

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

lemmatizer = LemmatizerModel().pretrained().setInputCols(['corrected']).setOutputCol('lemma')

normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)

stop_wrd = list( stopwords.words('english'))

stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)
# bigram = NGramGenerator().setInputCols(["cleanTokens"]).setOutputCol("bigram").setN(2).setEnableCumulative(False)
trigram = NGramGenerator().setInputCols(["cleanTokens"]).setOutputCol("trigram").setN(3).setEnableCumulative(False) 

finisher = Finisher().setInputCols(['cleanTokens','trigram']).setOutputCols(['uni','tri']).setOutputAsArray(True)

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [11]:
pipeline_bow = Pipeline().setStages([
    assembler, tokenizer, spell_chk , 
    lemmatizer, normalizer,stop_words_cleaner,trigram,finisher
])
model_trans_tfIdf = pipeline_bow.fit(song_data)
model_trans_tfIdf =  model_trans_tfIdf.transform(song_data)
model_trans_tfIdf.show()

+--------------------+--------------------+-------+--------------------+--------------------+
|                 Key|                text|  Genre|                 uni|                 tri|
+--------------------+--------------------+-------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|[could feel time,...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|[take baby hold, ...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|[come baby yeah, ...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|[day youll rememb...|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|[grudge, slowly, ...|[grudge slowly we...|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|[donut, talk, lis...|[donut talk liste...|
|black veil brides...|Have we begun to ...|   Rock|[begin, drift, aw...|[begin drift away...|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|[aint, ne

In [12]:
# from pyspark.sql.functions import udf, explode
# from pyspark.ml.linalg import SparseVector, DenseVector
# from pyspark.sql.types import *

# @udf(ArrayType(StringType()))
# def concat_uni_bi_tri(uni,bi,tri):
#     join1 = uni + list(set(bi) - set(uni))
#     join2 = join1 + list(set(tri) - set(join1))
#     return join2

# model_trans_tfIdf = model_trans_tfIdf.select("text", concat_uni_bi_tri("uni","bi", "tri").alias("cleanTokens"))
# model_trans_tfIdf.show(10,truncate = False)


In [16]:
from pyspark.ml.feature import CountVectorizer, IDF
 
count_vectorizer = CountVectorizer(inputCol='tri', outputCol='tf', minDF=2)
 
#idf = IDF(inputCol='tf', outputCol='features', minDocFreq=10)
 
bow_pipeline = Pipeline().setStages([count_vectorizer])
 
bow_pipeline = bow_pipeline.fit(model_trans_tfIdf)
 
bows = bow_pipeline.transform(model_trans_tfIdf)
 
bows = bows.withColumnRenamed("tf","features") 
bows.show(10)

+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|                 Key|                text|Genre|                 uni|                 tri|            features|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...| Rock|[could, feel, tim...|[could feel time,...|(8814,[149,493,79...|
|10000 maniacs_Bec...|Take me now, baby...| Rock|[take, baby, hold...|[take baby hold, ...|(8814,[7,99,254,2...|
|jamiroquai_Rock D...|And it's coming a...|  Pop|[come, baby, yeah...|[come baby yeah, ...|(8814,[2469,7239]...|
|10000 maniacs_The...|These are. These ...| Rock|[day, youll, reme...|[day youll rememb...|(8814,[1456,1581,...|
|10000 maniacs_Eve...|Trudging slowly o...| Rock|[grudge, slowly, ...|[grudge slowly we...|(8814,[976,1985,2...|
|10000 maniacs_Don...|Don't talk, I wil...| Rock|[donut, talk, lis...|[donut talk liste...|(8814

In [17]:
bows.show(10)

+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|                 Key|                text|Genre|                 uni|                 tri|            features|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...| Rock|[could, feel, tim...|[could feel time,...|(8814,[149,493,79...|
|10000 maniacs_Bec...|Take me now, baby...| Rock|[take, baby, hold...|[take baby hold, ...|(8814,[7,99,254,2...|
|jamiroquai_Rock D...|And it's coming a...|  Pop|[come, baby, yeah...|[come baby yeah, ...|(8814,[2469,7239]...|
|10000 maniacs_The...|These are. These ...| Rock|[day, youll, reme...|[day youll rememb...|(8814,[1456,1581,...|
|10000 maniacs_Eve...|Trudging slowly o...| Rock|[grudge, slowly, ...|[grudge slowly we...|(8814,[976,1985,2...|
|10000 maniacs_Don...|Don't talk, I wil...| Rock|[donut, talk, lis...|[donut talk liste...|(8814

In [18]:
from pyspark.ml.clustering import LDA
 
num_topics = 6
 
lda = LDA(k=num_topics, maxIter=10)
 
model = lda.fit(bows)
 
#ll = model.logLikelihood(bows)
 
#lp = model.logPerplexity(bows)
 
#print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
 
#print("The upper bound on perplexity: " + str(lp))

In [37]:
lda_model = model.transform(bows)
lda_model.show()
lda_model.select('features','topicDistribution').show( truncate = False)

+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|                 Key|                text|  Genre|                 uni|                 tri|            features|   topicDistribution|
+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|[could feel time,...|(8814,[149,493,79...|[0.00257697960660...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|[take baby hold, ...|(8814,[7,99,254,2...|[0.00136849571612...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|[come baby yeah, ...|(8814,[2469,7239]...|[0.05227690376027...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|[day youll rememb...|(8814,[1456,1581,...|[0.00226980744860...|
|10000 maniacs_Eve...|Trudging slowly o...|   Ro

In [41]:
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
import numpy as np

# @udf(IntegerType())
@udf(StringType())
def dominant_topic(lst_per):
      if round(max(lst_per),2) == 0.0:
        k='NA'
      else:
        k =  lst_per.tolist().index(max(lst_per))
      # k = type(list(lst_per))
      return str(k)

# @udf(DoubleType())
@udf(StringType())
def percent_contri(lst_per):
      if round(max(lst_per),2) == 0.0:
        k = 'NA'
      else:
        k =  round(max(lst_per),2)
      # k = type(list(lst_per))
      return str(k)

dom_topic_model = lda_model.select('Key','topicDistribution',dominant_topic("topicDistribution").alias("dom_topic"),percent_contri("topicDistribution").alias("dom_per"))
dom_topic_model.show()

+--------------------+--------------------+---------+-------+
|                 Key|   topicDistribution|dom_topic|dom_per|
+--------------------+--------------------+---------+-------+
|10000 maniacs_Mor...|[0.00253408651399...|        4|   0.99|
|10000 maniacs_Bec...|[0.00136849571612...|        3|   0.99|
|jamiroquai_Rock D...|[0.05227690376027...|        4|   0.74|
|10000 maniacs_The...|[0.00226980744860...|        1|   0.99|
|10000 maniacs_Eve...|[0.00276634801413...|        1|   0.99|
|10000 maniacs_Don...|[0.00119692501982...|        1|   0.99|
|black veil brides...|[0.96525782794928...|        0|   0.96|
|lynyrd skynyrd_I ...|[0.02577741591746...|        3|   0.87|
|10000 maniacs_Acr...|[0.00272719535635...|        2|   0.99|
|10000 maniacs_Pla...|[0.00211308935419...|        4|   0.99|
|10000 maniacs_Rai...|[0.00238563173616...|        3|   0.99|
|twista_Back 2 School|[0.01093732538989...|        5|   0.54|
|10000 maniacs_Ant...|[0.00172630591182...|        5|   0.99|
|10000 m

# Vectorization

## BoW Method

Create a rank column TF*IDF 
Explore TF option too, it might give us better results.

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF

count_vectorizer = CountVectorizer(inputCol='cleanTokens', outputCol='tf', minDF=10)
cvmodel = count_vectorizer.fit(model_trans_tfIdf)
idf = IDF(inputCol='tf', outputCol='idf')

# finisher_idf = Finisher().setInputCols(['features']).setOutputCols(['tfidf']) finisher is for sparkNLP not spark ML
bow_pipeline = Pipeline().setStages([count_vectorizer, idf])
bow_pipeline = bow_pipeline.fit(model_trans_tfIdf)

bows = bow_pipeline.transform(model_trans_tfIdf)
bows.show()

+--------------------+--------------------+-------+--------------------+--------------------+--------------------+
|                 Key|                text|  Genre|         cleanTokens|                  tf|                 idf|
+--------------------+--------------------+-------+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|(882,[0,1,3,4,8,1...|(882,[0,1,3,4,8,1...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|(882,[0,6,7,10,12...|(882,[0,6,7,10,12...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|(882,[0,1,2,3,4,5...|(882,[0,1,2,3,4,5...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|(882,[1,9,11,15,1...|(882,[1,9,11,15,1...|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|[grudge, slowly, ...|(882,[4,12,24,80,...|(882,[4,12,24,80,...|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|[donut, talk, lis...|(882,[0,

Calculate TF-IDF Rank

Filter out row where length of TF and IDF is not same

In [ ]:
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *

@udf(ArrayType(IntegerType()))
def TF_indices(tf):
      K =  tf.indices.tolist()
      return K

@udf(ArrayType(DoubleType()))
def TF_value(tf):
   if isinstance(tf, SparseVector):
      K =  tf.values.tolist()
      return K

@udf(ArrayType(IntegerType()))
def IDF_indices(idf):
      K =  idf.indices.tolist()
      return K

@udf(ArrayType(DoubleType()))
def IDF_value(idf):
   if isinstance(idf, SparseVector):
      K =  idf.values.tolist()
      return K

@udf(IntegerType())
def TF_len(tf):
      K =  len(tf.indices.tolist())
      return K

@udf(IntegerType())
def IDF_len(idf):
      K =  len(idf.indices.tolist())
      return K

bows_Val = bows.select("Key","text",'Genre', TF_indices("tf").alias("TF_Ind"), TF_value("tf").alias("TF_Val"),IDF_indices("idf").alias("IDF_Ind"), IDF_value("idf").alias("IDF_Val"),TF_len("tf").alias("TF_Len"),IDF_len("idf").alias("IDF_Len") )
bows_Val.show(truncate = True)


+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+------+-------+
|                 Key|                text|  Genre|              TF_Ind|              TF_Val|             IDF_Ind|             IDF_Val|TF_Len|IDF_Len|
+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+------+-------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[0, 1, 3, 4, 8, 1...|[1.0, 7.0, 1.0, 1...|[0, 1, 3, 4, 8, 1...|[0.58364677447042...|    29|     30|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[0, 6, 7, 10, 12,...|[1.0, 1.0, 4.0, 1...|[0, 6, 7, 10, 12,...|[0.58364677447042...|    44|     44|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[0, 1, 2, 3, 4, 5...|[5.0, 1.0, 2.0, 1...|[0, 1, 2, 3, 4, 5...|[2.91823387235212...|    82|     81|
|10000 maniacs_The...|These are. These ...|   Rock|[1, 9, 11, 15, 16...|[6.0, 2.0, 2.0, 2...|[

In [ ]:
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *

@udf(ArrayType(DoubleType()))
def TFIDF_Rank(tf_val,idf_val):
    tf_val = [float(i) for i in tf_val]
    idf_val = [float(i) for i in idf_val]
    products = [a * b for a, b in zip(tf_val, idf_val)]
    return products

bows_Mul = bows_Val.select("Key","text", "Genre", TFIDF_Rank("TF_Val","IDF_Val").alias("TF_IDF_Rank"),"IDF_Ind")
bows_Mul.show(truncate = True)
    

+--------------------+--------------------+-------+--------------------+--------------------+
|                 Key|                text|  Genre|         TF_IDF_Rank|             IDF_Ind|
+--------------------+--------------------+-------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[0.58364677447042...|[0, 1, 3, 4, 8, 1...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[0.58364677447042...|[0, 6, 7, 10, 12,...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[14.5911693617606...|[0, 1, 2, 3, 4, 5...|
|10000 maniacs_The...|These are. These ...|   Rock|[20.1756564219314...|[1, 9, 11, 15, 16...|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|[7.94232143632533...|[4, 12, 24, 80, 1...|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|[0.58364677447042...|[0, 1, 3, 5, 9, 1...|
|black veil brides...|Have we begun to ...|   Rock|[0.75623115598652...|[2, 3, 4, 6, 8, 1...|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|[5.252820

In [ ]:
@udf(ArrayType(IntegerType()))
# @udf(StringType())
def Top3(rank,ind):
    rank = [float(i) for i in rank]
    ind = [int(i) for i in ind]
    top_ind = sorted(range(len(rank)), key=lambda i: rank[i])[-3:] # this give indices where we have top 3 values in rank
    top_wrd = [ind[i] for i in top_ind]
    return top_wrd

bows_wrdlevel = bows_Mul.select("Key","Genre", Top3("TF_IDF_Rank","IDF_Ind").alias("top_ind"))
bows_wrdlevel.show()

+--------------------+-------+---------------+
|                 Key|  Genre|        top_ind|
+--------------------+-------+---------------+
|10000 maniacs_Mor...|   Rock|    [1, 70, 59]|
|10000 maniacs_Bec...|   Rock| [424, 54, 319]|
|jamiroquai_Rock D...|    Pop|[294, 249, 588]|
|10000 maniacs_The...|   Rock|  [174, 32, 61]|
|10000 maniacs_Eve...|   Rock|[677, 775, 411]|
|10000 maniacs_Don...|   Rock| [169, 44, 107]|
|black veil brides...|   Rock|   [13, 147, 6]|
|lynyrd skynyrd_I ...|   Rock|    [33, 6, 41]|
|10000 maniacs_Acr...|   Rock|[226, 496, 297]|
|10000 maniacs_Pla...|   Rock| [23, 832, 195]|
|10000 maniacs_Rai...|   Rock|  [147, 53, 32]|
|twista_Back 2 School|Hip Hop| [24, 188, 375]|
|10000 maniacs_Ant...|   Rock| [678, 760, 95]|
|10000 maniacs_All...|   Rock| [163, 155, 65]|
|10000 maniacs_Bac...|   Rock|[370, 692, 351]|
|cyndi lauper_True...|    Pop| [450, 11, 438]|
|10000 maniacs_A R...|   Rock|  [65, 251, 63]|
|rick astley_She M...|    Pop|   [18, 15, 30]|
|steve earle_

## Approach 1: Top 3 > Embeddings

Vectorization create vector column object and we need to retrieve values and indices accordingly from it unlike python array/list
This whole para shows how we can retrieve indices and ranking from feature column in various ways by applying UDF

Convert indices to word

In [ ]:
from pyspark.sql import functions as F
from operator import itemgetter

vocab = cvmodel.vocabulary
# udf_to_words = udf(get_words, ArrayType(StringType()))
# use @udf if you have used that pattern, throughout your pynb
@udf(ArrayType(StringType()))
def get_words(token_in):
    return itemgetter(*token_in)(vocab)
    # return map(vocab.__getitem__, token_in)
    # return vocab[token_in].tolist()


This gives us top 3 word for each doc

In [ ]:
bow_top3_wrd = bows_wrdlevel.select('Key','Genre',get_words(F.col('top_ind')).alias('word'))
bow_top3_wrd.na.drop(subset=["word"])
bow_top3_wrd.show()
# bow_top5_wrd.schema


+--------------------+-------+--------------------+
|                 Key|  Genre|                word|
+--------------------+-------+--------------------+
|10000 maniacs_Mor...|   Rock|[know, nothing, t...|
|10000 maniacs_Bec...|   Rock|[lover, night, si...|
|jamiroquai_Rock D...|    Pop| [canst, mile, page]|
|10000 maniacs_The...|   Rock|  [true, day, youll]|
|10000 maniacs_Eve...|   Rock|[silent, merchant...|
|10000 maniacs_Don...|   Rock|[people, donut, r...|
|black veil brides...|   Rock|     [one, save, oh]|
|lynyrd skynyrd_I ...|   Rock|    [need, oh, baby]|
|10000 maniacs_Acr...|   Rock|[part, across, star]|
|10000 maniacs_Pla...|   Rock|[life, remind, tr...|
|10000 maniacs_Rai...|   Rock|  [save, would, day]|
|twista_Back 2 School|Hip Hop|    [back, use, hes]|
|10000 maniacs_Ant...|   Rock|[friend, searchin...|
|10000 maniacs_All...|   Rock|[sorry, hold, for...|
|10000 maniacs_Bac...|   Rock|[second, storm, y...|
|cyndi lauper_True...|    Pop|[see, true, outside]|
|10000 mania

### Embedding

embedding works on sentence as it tries to find out the relationship between words too and not just their count/occurence. Because of this we have Approach 2 where we are creating embedding first and then doing picking embeddings for top 3 words.

In [ ]:
from pyspark.sql.functions import udf, explode
#convert list to sentence
@udf( StringType())
def get_sentence(word_in):
    return " ".join(str(item) for item in word_in)

# udf_to_doc = udf(get_sentence, StringType())

bow_top_embd = bow_top3_wrd.select('Key','Genre', get_sentence(F.col('word')).alias('doc'))
# bow_top_embd = bow_top_embd.filter((F.col('doc') != '') | F.col('doc') != ' ')
bow_top_embd.na.drop(subset=["doc"])
from pyspark.sql.functions import trim
bow_top_embd = bow_top_embd.withColumn("doc", trim(bow_top_embd.doc))
# bow_top_embd.filter(!(F.col('doc') == ' ')).show()
bow_top_embd.show()

ModuleNotFoundError: ignored

In [ ]:
bow_top_embd.printSchema()
bow_top_embd.select('Genre').distinct().collect()

root
 |-- Key: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- doc: string (nullable = true)



[Row(Genre='Rock'), Row(Genre='Pop'), Row(Genre='Hip Hop')]

Embedding output here will be list of list 3*100 features

In [ ]:
assembler = DocumentAssembler().setInputCol('doc').setOutputCol("document").setCleanupMode("shrink")
tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')
embed = WordEmbeddingsModel.pretrained().setInputCols(['document',"token"]).setOutputCol("embeddings")
embeddings_finisher = EmbeddingsFinisher().setInputCols(["embeddings"]).setOutputCols(["embeddings"]).setOutputAsVector(True).setCleanAnnotations(False)
finisher = Finisher().setInputCols(['token']).setOutputCols(['token']).setOutputAsArray(True)
# vec_assembler = VectorAssembler().setInputCols(['embeddings']).setOutputCol('features')
pipeline_uni = Pipeline().setStages([
    assembler, tokenizer,embed ,embeddings_finisher, finisher
])
model = pipeline_uni.fit(bow_top_embd)
model_trans =  model.transform(bow_top_embd)#.persist()
# model_trans.select('embeddings').show(truncate = False)
model_trans.show(10)


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
+--------------------+-----+--------------------+--------------------+--------------------+
|                 Key|Genre|                 doc|               token|          embeddings|
+--------------------+-----+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...| Rock|nothing theres shout|[nothing, theres,...|[[0.1090399995446...|
|10000 maniacs_Bec...| Rock|  belong night lover|[belong, night, l...|[[0.0577719993889...|
|jamiroquai_Rock D...|  Pop|    coming page rock|[coming, page, rock]|[[0.0667089968919...|
|10000 maniacs_The...| Rock|    youll true bless|[youll, true, bless]|[[-0.556039988994...|
|10000 maniacs_Eve...| Rock|everyday pack silent|[everyday, pack, ...|[[-0.350219994783...|
|10000 maniacs_Don...| Rock|   really keep three|[really, keep, th...|[[-0.054349001497...|
|black veil brides...| Rock|     oh save mistake| [oh, save, mistake]|[[-0.

String Indexer to change string categories to label for classification

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Genre", outputCol = "label" ,)
pipeline = Pipeline(stages=[ label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(model_trans)
dataset = pipelineFit.transform(model_trans)
selectedCols = ['token','embeddings','label']
dataset = dataset.select(selectedCols)
dataset.show(5)
# dataset.printSchema()


+--------------------+--------------------+-----+
|               token|          embeddings|label|
+--------------------+--------------------+-----+
|[theres, nothing,...|[[-0.123620003461...|  0.0|
|[belong, night, l...|[[0.0577719993889...|  0.0|
|[coming, page, rock]|[[0.0667089968919...|  1.0|
|[youll, true, bless]|[[-0.556039988994...|  0.0|
|[everyday, pack, ...|[[-0.350219994783...|  0.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [ ]:
dataset.select('label').distinct().collect()

[Row(label=0.0), Row(label=1.0), Row(label=2.0)]

Spark ML takes vector as input so need to convert our embedding output to vector. We are going to use dense vector here. 

In [ ]:
import itertools
import functools
from pyspark.ml.linalg import Vectors, VectorUDT
@udf(VectorUDT())
# @udf(StringType())
def get_Flatvector(embd_vec):
  # embd_vec = list(embd_vec)
  merged = list(itertools.chain(*embd_vec))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  mer_floats = [float(np_float) for np_float in merged]
  ver = Vectors.dense(mer_floats)
  return ver
  # return str(type(mer_floats[0]))
  # return embd_vec[0].tolist()

dataset_clsf = dataset.select('token',get_Flatvector(F.col('embeddings')).alias('features'),'label')
dataset_clsf.na.drop(subset=["features"])
# dataset_clsf = dataset_clsf.where("'label' != ''").where("'label' != ' '")
# dataset_clsf = dataset_clsf.filter(dataset_clsf.label == 0.0)
dataset_clsf.show(truncate = True)

# dataset.select("label",flatten("embeddings")).show(false)

+--------------------+--------------------+-----+
|               token|            features|label|
+--------------------+--------------------+-----+
|[theres, nothing,...|[-0.1236200034618...|  0.0|
|[belong, night, l...|[0.05777199938893...|  0.0|
|[coming, page, rock]|[0.06670899689197...|  1.0|
|[youll, true, bless]|[-0.5560399889945...|  0.0|
|[everyday, pack, ...|[-0.3502199947834...|  0.0|
|[really, keep, th...|[-0.0543490014970...|  0.0|
| [oh, save, mistake]|[-0.6666499972343...|  0.0|
|    [baby, oh, need]|[0.49746999144554...|  0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|
|[brain, truth, st...|[-0.1024399995803...|  0.0|
|[theyve, save, wo...|[0.0,0.0,0.0,0.0,...|  0.0|
| [school, use, rock]|[0.80335998535156...|  2.0|
|[heat, fire, surr...|[-0.8194400072097...|  0.0|
|[forever, hold, s...|[-0.0296199992299...|  0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|
|[mama, true, afraid]|[0.27950999140739...|  1.0|
|   [wait, room, dry]|[-0.1498499959707...|  0.0|


We need to ensure that the size of feature is 300. group by count of feature to check and remove unwanted rows. This is probably because TFIDF gave same rank to multiple words and we have more than 3 in some cases. in our case it was just one row

In [ ]:
from pyspark.sql import functions as F
import itertools
import functools
from pyspark.ml.linalg import Vectors, VectorUDT
# @udf(VectorUDT())
@udf(IntegerType())
def get_Flatvector_len(embd_vec):
  # embd_vec = list(embd_vec)
  # merged = list(itertools.chain(*embd_vec))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  vec =Vectors.dense(embd_vec)
  return (len(vec))
  # return str(type(mer_floats[0]))
  # return embd_vec[0].tolist()

dataset_clsf_300 = dataset_clsf.select('token','features',get_Flatvector_len(F.col('features')).alias('feat_cnt'),'label')
dataset_clsf_300.na.drop(subset=["feat_cnt"])
dataset_clsf_300.na.drop(subset=["features"])
dataset_clsf_300.na.drop(subset=["label"])
# dataset_clsf.show(truncate = True)
# dataset_clsf = dataset_clsf.groupBy("features").agg(F.count('features'))
dataset_clsf_300 = dataset_clsf_300.filter((dataset_clsf_300.feat_cnt == 300) | (dataset_clsf_300.label != ' ') ).select('token','features','label')
# dataset_clsf_300 = dataset_clsf_300.filter(dataset_clsf_300.label != ' ').select('features','label')
# dataset_clsf_300 = dataset_clsf_300.filter(F.col('features') != ' ')
# dataset_clsf_300.show(dataset_clsf_300.count() )
# dataset_clsf_300.repartition(1).count()
dataset_clsf_300.show()
# dataset_clsf_300.count()
# dataset_clsf_300.groupBy("feat_cnt").agg(F.count('feat_cnt')).show()

+--------------------+--------------------+-----+
|               token|            features|label|
+--------------------+--------------------+-----+
|[theres, nothing,...|[-0.1236200034618...|  0.0|
|[belong, night, l...|[0.05777199938893...|  0.0|
|[coming, page, rock]|[0.06670899689197...|  1.0|
|[youll, true, bless]|[-0.5560399889945...|  0.0|
|[everyday, pack, ...|[-0.3502199947834...|  0.0|
|[really, keep, th...|[-0.0543490014970...|  0.0|
| [oh, save, mistake]|[-0.6666499972343...|  0.0|
|    [baby, oh, need]|[0.49746999144554...|  0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|
|[brain, truth, st...|[-0.1024399995803...|  0.0|
|[theyve, save, wo...|[0.0,0.0,0.0,0.0,...|  0.0|
| [school, use, rock]|[0.80335998535156...|  2.0|
|[heat, fire, surr...|[-0.8194400072097...|  0.0|
|[forever, hold, s...|[-0.0296199992299...|  0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|
|[mama, true, afraid]|[0.27950999140739...|  1.0|
|   [wait, room, dry]|[-0.1498499959707...|  0.0|


In [ ]:
# set seed for reproducibility
# train, test = dataset_clsf_300.randomSplit(weights=[0.6, 0.4], seed=100)
# dataset_clsf_300 = dataset_clsf_300.where("'features' != ''")
(trainingData, testData) = dataset_clsf_300.select('token','features','label').randomSplit([0.7, 0.3], seed = 100)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))
# train, test = dataset_clsf_300.randomSplit([0.6, 0.4], seed=100)
trainingData.show(10)

+--------------------+--------------------+-----+
|               token|            features|label|
+--------------------+--------------------+-----+
|      [act, eye, uh]|[0.24027000367641...|  0.0|
|[afraid, perfect,...|[0.20393000543117...|  0.0|
|[afraid, perfect,...|[0.20393000543117...|  0.0|
|[afraid, sweet, p...|[0.20393000543117...|  0.0|
|   [ah, baby, steal]|[-1.4133000373840...|  1.0|
|[ah, believe, fri...|[-1.4133000373840...|  0.0|
| [alone, goin, help]|[0.38927999138832...|  0.0|
|[alone, wrong, ni...|[0.38927999138832...|  0.0|
|[already, round, ...|[0.13563999533653...|  0.0|
|[already, round, ...|[0.13563999533653...|  0.0|
+--------------------+--------------------+-----+
only showing top 10 rows



### Classification

### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label',maxIter=20, regParam=0.3, elasticNetParam=0   )
lrModel = lr.fit(trainingData )
predictions = lrModel.transform(testData)
predictions.show()

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|               token|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
| [ago, rock, become]|[0.54666000604629...|  0.0|[1.41761643875231...|[0.79741206092989...|       0.0|
|   [ah, yeah, crazy]|[-1.4133000373840...|  1.0|[-0.1186564800670...|[0.24700552033410...|       1.0|
|[aint, call, always]|[-0.1444900035858...|  1.0|[1.66194859571792...|[0.83348033887642...|       0.0|
|[alive, night, ever]|[-0.0055056000128...|  1.0|[1.50112892351291...|[0.80985501917549...|       0.0|
|[already, young, ...|[0.13563999533653...|  0.0|[1.28606549779604...|[0.75176838242710...|       0.0|
|  [alright, oh, try]|[0.34696000814437...|  1.0|[1.30015038665708...|[0.77115126204828...|       0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|[1.67267610797735...|[0.

In [ ]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)
# predictions.show()

0.735822417115046

#### Top Words

In [ ]:
top_wrd_df = predictions.filter(predictions.label == predictions.prediction)
top_wrd_df.show()
top_wrd_df.count()

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|               token|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
| [ago, rock, become]|[0.54666000604629...|  0.0|[1.41761643875231...|[0.79741206092989...|       0.0|
|   [ah, yeah, crazy]|[-1.4133000373840...|  1.0|[-0.1186564800670...|[0.24700552033410...|       1.0|
|[already, young, ...|[0.13563999533653...|  0.0|[1.28606549779604...|[0.75176838242710...|       0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|[1.67267610797735...|[0.82726193161393...|       0.0|
| [angel, today, gun]|[0.34995999932289...|  0.0|[1.45533061587012...|[0.78274042762116...|       0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|[1.94382655069015...|[0.85622207318998...|       0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|[1.94382655069015...|[0.

220

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(top_wrd_df['label']).orderBy(top_wrd_df['probability'].desc())

top_wrd_df = top_wrd_df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10)
top_wrd_df.count()

21

In [ ]:
top_wrd_df_lst = top_wrd_df.groupBy('label').agg(F.collect_list("token"))
top_wrd_df_lst.show(truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|collect_list(token)                                                                                                                                                                                                          |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |[[human, mistake, watch], [put, fire, set], [spit, regret, round], [screen, yesterday, young], [stay, lot, till], [inside, deep, head], [fix, plain, thin], [black, fate, end], [circle, line, stone], [mirror, clear, away]]|
|1.0  |[[oh, wasnt, cry], [love, hello, feelings], [oh, end, girl], [oh, care, w

In [ ]:
# trainingSummary = lrModel.summary
# roc = trainingSummary.roc.toPandas()
# plt.plot(roc['FPR'],roc['TPR'])
# plt.ylabel('False Positive Rate')
# plt.xlabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.show()
# print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

### Naive Bayes

We can only use Naive Bayes for positive values, we have negative too.

In [ ]:
# from pyspark.ml.classification import NaiveBayes
# nb = NaiveBayes(smoothing=1)
# model = nb.fit(trainingData)
# predictions = model.transform(testData)
# # predictions.filter(predictions['prediction'] == 0) \
# #     .select("Descript","Category","probability","label","prediction") \
# #     .orderBy("probability", ascending=False) \
# #     .show(n = 10, truncate = 30)
# predictions.show()

### Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
algo = RandomForestClassifier(featuresCol='features', labelCol='label')
model = algo.fit(trainingData)
predictions = model.transform(testData)
predictions.show()


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accracy = evaluator.evaluate(predictions)
accracy

In [ ]:
# #important: need to cast to float type, and order by prediction, else it won't work
# preds_and_labels = predictions.select(['prediction','d']).withColumn('label', F.col('d').cast(FloatType())).orderBy('prediction')

# #select only prediction and label columns
# preds_and_labels = preds_and_labels.select(['prediction','label'])

# metrics = MultiClassMetrics(preds_and_labels.rdd.map(tuple))

y_true = predictions.select(['label']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

## Approach 2: Embeddings > Top 3

Start with creating embeddings first

In [ ]:
assembler = DocumentAssembler().setInputCol('text').setOutputCol('document')

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

lemmatizer = LemmatizerModel().pretrained().setInputCols(['corrected']).setOutputCol('lemma')

normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)

stop_wrd = list( stopwords.words('english'))

stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)

embed = WordEmbeddingsModel.pretrained().setInputCols(['document',"token"]).setOutputCol("embeddings")
embeddings_finisher = EmbeddingsFinisher().setInputCols(["embeddings"]).setOutputCols(["embeddings"]).setOutputAsVector(True).setCleanAnnotations(False)

finisher = Finisher().setInputCols(['cleanTokens']).setOutputCols(['cleanTokens']).setOutputAsArray(True).setCleanAnnotations(True)

pipeline_embd = Pipeline().setStages([ assembler, tokenizer, spell_chk , lemmatizer, normalizer,stop_words_cleaner,embed,embeddings_finisher,finisher ]) 
model_trans_tfIdf_2 = pipeline_embd.fit(song_data) 
model_trans_tfIdf_2 = model_trans_tfIdf_2.transform(song_data) 
model_trans_tfIdf_2.show(10)
# model_trans_tfIdf_2.select('embeddings').show(10,truncate = False)

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
+--------------------+--------------------+-----+--------------------+--------------------+
|                 Key|                text|Genre|         cleanTokens|          embeddings|
+--------------------+--------------------+-----+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...| Rock|[could, feel, tim...|[[-0.046539001166...|
|10000 maniacs_Bec...|Take me now, baby...| Rock|[take, baby, hold...|[[-0.270639985799...|
|jamiroquai_Rock D...|And it's coming a...|  Pop|[come, baby, yeah...|[[-0.071952998638...|
|10000 maniacs_The...|These are. These ...| Rock|[day, youll, reme...|[[-0.604569971561...|
|10000 maniacs_Eve...|Trudging slowly o...| Rock|[grudge, 

1. Create TFIDF
2. pick top 3 position where rank is max
3. pick embeddings for that top 3 position
4. map it to original words

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF

count_vectorizer = CountVectorizer(inputCol='cleanTokens', outputCol='tf', minDF=10)
cvmodel = count_vectorizer.fit(model_trans_tfIdf_2)
idf = IDF(inputCol='tf', outputCol='idf', minDocFreq=10)
# finisher_idf = Finisher().setInputCols(['features']).setOutputCols(['tfidf']) finisher is for sparkNLP not spark ML
bow_pipeline_2 = Pipeline().setStages([count_vectorizer, idf])
bow_pipeline_2 = bow_pipeline_2.fit(model_trans_tfIdf_2)

bows_2 = bow_pipeline_2.transform(model_trans_tfIdf_2)
bows_2.show()

+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|                 Key|                text|  Genre|         cleanTokens|          embeddings|                  tf|                 idf|
+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|[[-0.046539001166...|(882,[0,1,3,4,8,1...|(882,[0,1,3,4,8,1...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|[[-0.270639985799...|(882,[0,6,7,10,12...|(882,[0,6,7,10,12...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|[[-0.071952998638...|(882,[0,1,2,3,4,5...|(882,[0,1,2,3,4,5...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|[[-0.604569971561...|(882,[1,9,11,15,1...|(882,[1,9,11,15,1...|
|10000 maniacs_Eve...|Trudging slowly o...|   Ro

In [ ]:
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *

@udf(ArrayType(IntegerType()))
def TF_indices(tf):
      K =  tf.indices.tolist()
      return K

@udf(ArrayType(DoubleType()))
def TF_value(tf):
   if isinstance(tf, SparseVector):
      K =  tf.values.tolist()
      return K

@udf(ArrayType(IntegerType()))
def IDF_indices(idf):
      K =  idf.indices.tolist()
      return K

@udf(ArrayType(DoubleType()))
def IDF_value(idf):
   if isinstance(idf, SparseVector):
      K =  idf.values.tolist()
      return K

@udf(IntegerType())
def TF_len(tf):
      K =  len(tf.indices.tolist())
      return K

@udf(IntegerType())
def IDF_len(idf):
      K =  len(idf.indices.tolist())
      return K

bows_Val2 = bows_2.select("Key",'Genre',"embeddings", TF_indices("tf").alias("TF_Ind"), TF_value("tf").alias("TF_Val"),IDF_indices("idf").alias("IDF_Ind"), IDF_value("idf").alias("IDF_Val"),TF_len("tf").alias("TF_Len"),IDF_len("idf").alias("IDF_Len") )
bows_Val2.show(truncate = True)

+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-------+
|                 Key|  Genre|          embeddings|              TF_Ind|              TF_Val|             IDF_Ind|             IDF_Val|TF_Len|IDF_Len|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-------+
|10000 maniacs_Mor...|   Rock|[[-0.046539001166...|[0, 1, 3, 4, 8, 1...|[1.0, 7.0, 1.0, 1...|[0, 1, 3, 4, 8, 1...|[0.58364677447042...|    29|     29|
|10000 maniacs_Bec...|   Rock|[[-0.270639985799...|[0, 6, 7, 10, 12,...|[1.0, 1.0, 4.0, 1...|[0, 6, 7, 10, 12,...|[0.58364677447042...|    44|     44|
|jamiroquai_Rock D...|    Pop|[[-0.071952998638...|[0, 1, 2, 3, 4, 5...|[5.0, 1.0, 2.0, 1...|[0, 1, 2, 3, 4, 5...|[2.91823387235212...|    82|     81|
|10000 maniacs_The...|   Rock|[[-0.604569971561...|[1, 9, 11, 15, 16...|[6.0, 2.0, 2.0, 2...|[

In [ ]:
# from pyspark.sql.functions import udf, explode
# from pyspark.ml.linalg import SparseVector, DenseVector
# from pyspark.sql.types import *

@udf(ArrayType(DoubleType()))
def TFIDF_Rank(tf_val,idf_val):
    tf_val = [float(i) for i in tf_val]
    idf_val = [float(i) for i in idf_val]
    products = [a * b for a, b in zip(tf_val, idf_val)]
    return products

bows_Mul2 = bows_Val2.select("Key", "Genre", "embeddings",TFIDF_Rank("TF_Val","IDF_Val").alias("TF_IDF_Rank"),"IDF_Ind")
bows_Mul2.show(truncate = True)

+--------------------+-------+--------------------+--------------------+--------------------+
|                 Key|  Genre|          embeddings|         TF_IDF_Rank|             IDF_Ind|
+--------------------+-------+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...|   Rock|[[-0.046539001166...|[0.58364677447042...|[0, 1, 3, 4, 8, 1...|
|10000 maniacs_Bec...|   Rock|[[-0.270639985799...|[0.58364677447042...|[0, 6, 7, 10, 12,...|
|jamiroquai_Rock D...|    Pop|[[-0.071952998638...|[14.5911693617606...|[0, 1, 2, 3, 4, 5...|
|10000 maniacs_The...|   Rock|[[-0.604569971561...|[20.1756564219314...|[1, 9, 11, 15, 16...|
|10000 maniacs_Eve...|   Rock|[[-0.136580005288...|[7.94232143632533...|[4, 12, 24, 80, 1...|
|10000 maniacs_Don...|   Rock|[[0.0,0.0,0.0,0.0...|[0.58364677447042...|[0, 1, 3, 5, 9, 1...|
|black veil brides...|   Rock|[[0.1571100056171...|[0.75623115598652...|[2, 3, 4, 6, 8, 1...|
|lynyrd skynyrd_I ...|   Rock|[[0.0,0.0,0.0,0.0...|[5.252820

In [ ]:
@udf(ArrayType(IntegerType()))
# # @udf(StringType())
def Top3_pos(rank,ind):
    rank = [float(i) for i in rank]
    ind = [int(i) for i in ind]
    top_ind = sorted(range(len(rank)), key=lambda i: rank[i])[-3:] # this give indices where we have top 3 values in rank
    # top_wrd = [ind[i] for i in top_ind]
    return top_ind

@udf(ArrayType(IntegerType()))
def Top3_wrd(rank,ind):
    rank = [float(i) for i in rank]
    ind = [int(i) for i in ind]
    top_ind = sorted(range(len(rank)), key=lambda i: rank[i])[-3:] # this give indices where we have top 3 values in rank
    top_wrd = [ind[i] for i in top_ind]
    return top_wrd

bows_wrdlevel_2 = bows_Mul2.select("Key","Genre","embeddings", Top3_pos("TF_IDF_Rank","IDF_Ind").alias("top3_pos"),Top3_wrd("TF_IDF_Rank","IDF_Ind").alias("top3_wrd"))
bows_wrdlevel_2.show()

+--------------------+-------+--------------------+------------+---------------+
|                 Key|  Genre|          embeddings|    top3_pos|       top3_wrd|
+--------------------+-------+--------------------+------------+---------------+
|10000 maniacs_Mor...|   Rock|[[-0.046539001166...| [1, 16, 17]|    [1, 59, 70]|
|10000 maniacs_Bec...|   Rock|[[-0.270639985799...|[33, 13, 30]| [424, 54, 321]|
|jamiroquai_Rock D...|    Pop|[[-0.071952998638...|[74, 50, 75]|[585, 249, 608]|
|10000 maniacs_The...|   Rock|[[-0.604569971561...|  [6, 19, 8]|  [32, 174, 61]|
|10000 maniacs_Eve...|   Rock|[[-0.136580005288...|[18, 20, 11]|[677, 767, 413]|
|10000 maniacs_Don...|   Rock|[[0.0,0.0,0.0,0.0...|[41, 19, 34]| [169, 45, 107]|
|black veil brides...|   Rock|[[0.1571100056171...|  [5, 15, 3]|   [13, 147, 6]|
|lynyrd skynyrd_I ...|   Rock|[[0.0,0.0,0.0,0.0...| [16, 4, 20]|    [33, 6, 41]|
|10000 maniacs_Acr...|   Rock|[[-0.530860006809...|[24, 31, 27]|[226, 490, 298]|
|10000 maniacs_Pla...|   Roc

Select embeddings at those top 3 positions

In [ ]:
from collections import Counter
from pyspark.sql.functions import udf, explode
from pyspark.sql.functions import struct, array
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, VectorUDT


# @udf(ArrayType(ArrayType(FloatType())))
# @udf(ArrayType(FloatType()))
# @udf(StringType())
# @udf(IntegerType())
@udf(VectorUDT())
def get_embd_top3(indices,embedd):
  
  # merged = list(itertools.chain(*embedd))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  # ver = Vectors.dense(mer_floats)
  #create list of list from embedding using positions
  # indices = [int(i) for i in indices]
  # embedd = v[1]
  embd_top3 = []
  
  for i in indices:
    sublist = embedd[i].values.tolist()
    sublist = [float(np_float) for np_float in sublist]
    embd_top3.append(sublist)
  
  #flatten out the list
  # merged = list(itertools.chain.from_iterable(embd_top3))
  merged = [j for sub in embd_top3 for j in sub]
  
  #convert to vector and send
  ver = Vectors.dense(merged)

  # tp = type(merged)
  return ver
  # return str(type(ver))
  # return len(merged) 

bows_embed_2 = bows_wrdlevel_2.select("Key","Genre", get_embd_top3("top3_pos", "embeddings").alias("features"))
# bows_embed_2.na.drop(subset=["features"])
bows_embed_2.show(truncate = False)
# bows_wrdlevel_2.select('embeddings').show(truncate = False)

+-------------------------------------+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

1. filter out column != 300
2. stringlabel indexer

In [ ]:
from pyspark.sql import functions as F
import itertools
import functools
from pyspark.ml.linalg import Vectors, VectorUDT
# @udf(VectorUDT())
@udf(IntegerType())
def get_Flatvector_len_2(embd_vec):
  # embd_vec = list(embd_vec)
  # merged = list(itertools.chain(*embd_vec))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  vec =Vectors.dense(embd_vec)
  return (len(vec))
  # return str(type(mer_floats[0]))
  # return embd_vec[0].tolist()

dataset_clsf_300_2 = bows_embed_2.select('features',get_Flatvector_len_2(F.col('features')).alias('feat_cnt'),'Genre')
dataset_clsf_300_2.na.drop(subset=["feat_cnt"])
dataset_clsf_300_2.na.drop(subset=["features"])
# dataset_clsf_300.na.drop(subset=["label"])
# dataset_clsf.show(truncate = True)
# dataset_clsf = dataset_clsf.groupBy("features").agg(F.count('features'))
dataset_clsf_300_2 = dataset_clsf_300_2.filter((dataset_clsf_300_2.feat_cnt == 300) | (dataset_clsf_300_2.Genre != ' ') ).select('features','Genre')
# dataset_clsf_300 = dataset_clsf_300.filter(dataset_clsf_300.label != ' ').select('features','label')
# dataset_clsf_300 = dataset_clsf_300.filter(F.col('features') != ' ')
# dataset_clsf_300.show(dataset_clsf_300.count() )
# dataset_clsf_300.repartition(1).count()

dataset_clsf_300_2.show()
# dataset_clsf_300.groupBy("feat_cnt").agg(F.count('feat_cnt')).show()

+--------------------+-------+
|            features|  Genre|
+--------------------+-------+
|[0.05869000032544...|   Rock|
|[-0.2186300009489...|   Rock|
|[-0.1416600048542...|    Pop|
|[-0.0381940007209...|   Rock|
|[-0.2708599865436...|   Rock|
|[0.05423000082373...|   Rock|
|[-0.1037900000810...|   Rock|
|[-0.1810200065374...|   Rock|
|[-0.2100400030612...|   Rock|
|[-0.5455800294876...|   Rock|
|[-0.3668900132179...|   Rock|
|[-0.3521400094032...|Hip Hop|
|[-0.1440100073814...|   Rock|
|[0.31435999274253...|   Rock|
|[0.34220001101493...|   Rock|
|[-0.3397899866104...|    Pop|
|[0.17934000492095...|   Rock|
|[-0.4695299863815...|    Pop|
|[0.71305000782012...|   Rock|
|[-0.0719529986381...|   Rock|
+--------------------+-------+
only showing top 20 rows



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Genre", outputCol = "label" ,)
pipeline = Pipeline(stages=[ label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(dataset_clsf_300_2)
dataset = pipelineFit.transform(dataset_clsf_300_2)
selectedCols = ['features','label']
dataset = dataset.select(selectedCols)
dataset.show(5)
dataset.select('label').distinct().collect()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.05869000032544...|  0.0|
|[-0.2186300009489...|  0.0|
|[-0.2163199931383...|  1.0|
|[-0.0381940007209...|  0.0|
|[-0.2708599865436...|  0.0|
+--------------------+-----+
only showing top 5 rows



[Row(label=0.0), Row(label=1.0), Row(label=2.0)]

continue classification from here.............................

# Approach 3: Create Labels (unsuprevised) > Topic Modeling